## Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

In [2]:
#training data
#inputs -> temp, humidity, rainfall
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 76, 90]], dtype='float32')

#targets -> apples, oranges
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [3]:
#inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

#### Model

In [4]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.0215, -0.7179,  0.4650],
        [-0.1398,  1.0408, -0.5982]], requires_grad=True)
tensor([-0.8385, -0.1636], requires_grad=True)


In [5]:
#model
def model(x):
    return x @ w.t() + b        #@ -> matrix multiplication in pytorch; t-> transpose

In [6]:
#predictions
preds = model(inputs)
print(preds)

tensor([[-27.3684,  33.6459],
        [-32.2907,  40.4257],
        [-68.1886,  92.4525],
        [-12.3058,   8.2004],
        [-12.0590,  15.4591]], grad_fn=<AddBackward0>)


In [7]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
#loss function (mse)
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [9]:
loss = mse(preds, targets)
loss.sqrt()

tensor(93.5014, grad_fn=<SqrtBackward0>)

In [10]:
loss.backward()

In [11]:
print(w)
print(w.grad)

tensor([[ 0.0215, -0.7179,  0.4650],
        [-0.1398,  1.0408, -0.5982]], requires_grad=True)
tensor([[ -8823.8027, -10171.6338,  -6663.4014],
        [ -4355.1270,  -4461.4263,  -3635.2021]])


In [12]:
#resetting gradients back to 0 to prevent adding up
w.grad.zero_()
b.grad.zero_()
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


##### Gradient Descent

In [13]:
preds = model(inputs)
preds

tensor([[-27.3684,  33.6459],
        [-32.2907,  40.4257],
        [-68.1886,  92.4525],
        [-12.3058,   8.2004],
        [-12.0590,  15.4591]], grad_fn=<AddBackward0>)

In [14]:
loss = mse(preds, targets)
loss

tensor(8742.5215, grad_fn=<DivBackward0>)

In [15]:
#compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -8823.8027, -10171.6338,  -6663.4014],
        [ -4355.1270,  -4461.4263,  -3635.2021]])
tensor([-106.6425,  -53.9633])


In [16]:
#adjust weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [17]:
#new weights and biases
print(w)
print(b)

tensor([[ 0.1098, -0.6161,  0.5317],
        [-0.0963,  1.0855, -0.5618]], requires_grad=True)
tensor([-0.8375, -0.1631], requires_grad=True)


In [18]:
#re-calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(6208.9624, grad_fn=<DivBackward0>)

In [19]:
#training with 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [20]:
#new loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(349.1335, grad_fn=<DivBackward0>)


In [21]:
preds

tensor([[ 60.8495,  75.3189],
        [ 85.9307,  98.7274],
        [ 92.1526, 152.7274],
        [ 53.0730,  47.1750],
        [102.1380,  83.9245]], grad_fn=<AddBackward0>)

In [22]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])